In [ ]:
import ast
import pandas as pd
import glob
import os

# ── 1. 메타 데이터 파싱 ──
# meta_Movies_and_TV.json 파일에서 각 줄마다 파이썬 딕셔너리 형태로 저장된 데이터를 읽어, 
# asin(상품ID)와 영화 제목(title)을 매핑하는 딕셔너리를 생성합니다.
meta_file_path = r"C:\Users\user\OneDrive\문서\과제\2025-1\캡스톤 디자인\meta_Movies_and_TV.json"

asin_to_title = {}

with open(meta_file_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            # 각 줄을 안전하게 파싱합니다.
            data = ast.literal_eval(line)
            asin = data.get('asin')
            title = data.get('title')
            if asin:
                asin_to_title[asin] = title
        except Exception as e:
            print(f"메타 데이터 파싱 오류 발생: {e}")

print(f"총 {len(asin_to_title)}개의 메타 데이터를 파싱했습니다.")

# ── 2. CSV 파일(Chunk) 처리 ──
# 각 파일은 예시로 movies_chunk_0.csv, movies_chunk_1.csv 등으로 저장되어 있다고 가정합니다.
csv_folder = r"C:\Users\user\OneDrive\문서\과제\2025-1\캡스톤 디자인\movies_chunk"
file_pattern = os.path.join(csv_folder, "movies_chunk_*.csv")
csv_files = glob.glob(file_pattern)

if not csv_files:
    print("처리할 CSV 파일이 없습니다.")
else:
    print("처리할 CSV 파일 목록:")
    for file in csv_files:
        print("  ", file)

# 각 CSV 파일을 순회하면서 처리합니다.
for file_path in csv_files:
    print(f"\n파일 처리 중: {file_path}")
    try:
        # 1) CSV 파일 읽기
        chunk_df = pd.read_csv(file_path)
        
        # 2) 'product/productId' 컬럼을 기준으로 영화 제목 추가
        #    meta 데이터의 asin과 매칭하여 새로운 'movie_title' 컬럼에 저장합니다.
        chunk_df['movie_title'] = chunk_df['product/productId'].map(asin_to_title)
        
        # 3) 결과를 새로운 CSV 파일로 저장 (파일명에 _with_title 추가)
        output_path = file_path.replace(".csv", "_with_title.csv")
        chunk_df.to_csv(output_path, index=False)
        print(f"저장 완료: {output_path}")
        
        # 메모리 관리를 위해 DataFrame 삭제 (필요 시)
        del chunk_df
        
    except Exception as e:
        print(f"{file_path} 처리 중 오류 발생: {e}")
